In [ ]:
from google.colab import files
import os
import shutil

input_dir = "data/vocals"
os.makedirs(input_dir, exist_ok=True)

print("📢 الرجاء رفع ملف صوتي بصيغة .wav")
uploaded = files.upload()

for name in uploaded.keys():
    dst = os.path.join(input_dir, name)
    shutil.move(name, dst)
    print(f"✅ تم حفظ الملف في: {dst}")


📢 الرجاء رفع ملف صوتي بصيغة .wav


Saving ahri voice.mp3 to ahri voice.mp3
✅ تم حفظ الملف في: data/vocals/ahri voice.mp3


In [ ]:
!pip install -q soundfile librosa


In [ ]:
import librosa
import soundfile
import numpy as np

class Slicer:
    def __init__(self, sr, threshold=-40., min_length=5000, min_interval=300, hop_size=20, max_sil_kept=5000):
        if not min_length >= min_interval >= hop_size:
            raise ValueError("min_length >= min_interval >= hop_size")
        if not max_sil_kept >= hop_size:
            raise ValueError("max_sil_kept >= hop_size")
        min_interval = sr * min_interval / 1000
        self.threshold = 10 ** (threshold / 20.)
        self.hop_size = round(sr * hop_size / 1000)
        self.win_size = min(round(min_interval), 4 * self.hop_size)
        self.min_length = round(sr * min_length / 1000 / self.hop_size)
        self.min_interval = round(min_interval / self.hop_size)
        self.max_sil_kept = round(sr * max_sil_kept / 1000 / self.hop_size)

    def _apply_slice(self, waveform, begin, end):
        return waveform[begin * self.hop_size: min(len(waveform), end * self.hop_size)]

    def slice(self, waveform):
        samples = waveform if waveform.ndim == 1 else waveform.mean(axis=0)
        if samples.shape[0] <= self.min_length:
            return [waveform]
        rms = librosa.feature.rms(y=samples, frame_length=self.win_size, hop_length=self.hop_size)[0]
        sil_tags, silence_start, clip_start = [], None, 0
        for i, r in enumerate(rms):
            if r < self.threshold:
                if silence_start is None:
                    silence_start = i
                continue
            if silence_start is None:
                continue
            if i - silence_start >= self.min_interval and i - clip_start >= self.min_length:
                pos = rms[silence_start:i + 1].argmin() + silence_start
                sil_tags.append((clip_start, pos))
                clip_start = pos
            silence_start = None
        total = rms.shape[0]
        if silence_start is not None and total - silence_start >= self.min_interval:
            pos = rms[silence_start:].argmin() + silence_start
            sil_tags.append((clip_start, pos))
        chunks = [self._apply_slice(waveform, s, e) for s, e in sil_tags]
        return chunks if chunks else [waveform]

non_silenced_dir = "data/non_silenced"
os.makedirs(non_silenced_dir, exist_ok=True)

for audio_file in os.listdir(input_dir):
    path = os.path.join(input_dir, audio_file)
    audio, sr = librosa.load(path, sr=None, mono=True)
    slicer = Slicer(sr=sr, threshold=-25, min_length=7000, min_interval=600, hop_size=15, max_sil_kept=700)
    chunks = slicer.slice(audio)
    for i, chunk in enumerate(chunks):
        save_path = os.path.join(non_silenced_dir, f"{audio_file.split('.')[0]}_{i}.wav")
        soundfile.write(save_path, chunk, sr)
        print(f"✅ مقطع بدون صمت: {save_path}")


✅ مقطع بدون صمت: data/non_silenced/ahri voice_0.wav
✅ مقطع بدون صمت: data/non_silenced/ahri voice_1.wav
✅ مقطع بدون صمت: data/non_silenced/ahri voice_2.wav
✅ مقطع بدون صمت: data/non_silenced/ahri voice_3.wav
✅ مقطع بدون صمت: data/non_silenced/ahri voice_4.wav
✅ مقطع بدون صمت: data/non_silenced/ahri voice_5.wav
✅ مقطع بدون صمت: data/non_silenced/ahri voice_6.wav
✅ مقطع بدون صمت: data/non_silenced/ahri voice_7.wav
✅ مقطع بدون صمت: data/non_silenced/ahri voice_8.wav
✅ مقطع بدون صمت: data/non_silenced/ahri voice_9.wav
✅ مقطع بدون صمت: data/non_silenced/ahri voice_10.wav
✅ مقطع بدون صمت: data/non_silenced/ahri voice_11.wav
✅ مقطع بدون صمت: data/non_silenced/ahri voice_12.wav
✅ مقطع بدون صمت: data/non_silenced/ahri voice_13.wav
✅ مقطع بدون صمت: data/non_silenced/ahri voice_14.wav
✅ مقطع بدون صمت: data/non_silenced/ahri voice_15.wav
✅ مقطع بدون صمت: data/non_silenced/ahri voice_16.wav
✅ مقطع بدون صمت: data/non_silenced/ahri voice_17.wav
✅ مقطع بدون صمت: data/non_silenced/ahri voice_18.wav
✅ م

In [ ]:
clip_dir = "data/clips"
os.makedirs(clip_dir, exist_ok=True)

min_sec = 6

from tqdm import tqdm
for file in tqdm(os.listdir(non_silenced_dir)):
    if file.endswith(".wav"):
        path = os.path.join(non_silenced_dir, file)
        audio, sr = librosa.load(path, sr=None)
        samples = int(min_sec * sr)
        i, idx = 0, 0
        while idx + samples < len(audio):
            segment = audio[idx:idx+samples]
            save_name = f"{file.split('.')[0]}_seg{i}.wav"
            soundfile.write(os.path.join(clip_dir, save_name), segment, sr)
            print(f"📁 segment saved: {save_name}")
            idx += samples
            i += 1


 23%|██▎       | 14/62 [00:00<00:00, 64.37it/s]

📁 segment saved: ahri voice_47_seg0.wav
📁 segment saved: ahri voice_53_seg0.wav
📁 segment saved: ahri voice_27_seg0.wav
📁 segment saved: ahri voice_54_seg0.wav
📁 segment saved: ahri voice_38_seg0.wav
📁 segment saved: ahri voice_6_seg0.wav
📁 segment saved: ahri voice_19_seg0.wav
📁 segment saved: ahri voice_33_seg0.wav
📁 segment saved: ahri voice_58_seg0.wav
📁 segment saved: ahri voice_35_seg0.wav
📁 segment saved: ahri voice_22_seg0.wav
📁 segment saved: ahri voice_2_seg0.wav
📁 segment saved: ahri voice_2_seg1.wav
📁 segment saved: ahri voice_30_seg0.wav
📁 segment saved: ahri voice_5_seg0.wav


 35%|███▌      | 22/62 [00:00<00:00, 68.17it/s]

📁 segment saved: ahri voice_41_seg0.wav
📁 segment saved: ahri voice_23_seg0.wav
📁 segment saved: ahri voice_7_seg0.wav
📁 segment saved: ahri voice_10_seg0.wav
📁 segment saved: ahri voice_48_seg0.wav
📁 segment saved: ahri voice_24_seg0.wav
📁 segment saved: ahri voice_60_seg0.wav
📁 segment saved: ahri voice_59_seg0.wav
📁 segment saved: ahri voice_9_seg0.wav
📁 segment saved: ahri voice_17_seg0.wav
📁 segment saved: ahri voice_52_seg0.wav
📁 segment saved: ahri voice_11_seg0.wav
📁 segment saved: ahri voice_39_seg0.wav
📁 segment saved: ahri voice_1_seg0.wav
📁 segment saved: ahri voice_45_seg0.wav


 61%|██████▏   | 38/62 [00:00<00:00, 70.17it/s]

📁 segment saved: ahri voice_44_seg0.wav
📁 segment saved: ahri voice_20_seg0.wav
📁 segment saved: ahri voice_28_seg0.wav
📁 segment saved: ahri voice_40_seg0.wav
📁 segment saved: ahri voice_29_seg0.wav
📁 segment saved: ahri voice_31_seg0.wav
📁 segment saved: ahri voice_0_seg0.wav
📁 segment saved: ahri voice_50_seg0.wav
📁 segment saved: ahri voice_16_seg0.wav
📁 segment saved: ahri voice_26_seg0.wav
📁 segment saved: ahri voice_25_seg0.wav
📁 segment saved: ahri voice_12_seg0.wav
📁 segment saved: ahri voice_42_seg0.wav
📁 segment saved: ahri voice_43_seg0.wav
📁 segment saved: ahri voice_4_seg0.wav


 87%|████████▋ | 54/62 [00:00<00:00, 70.72it/s]

📁 segment saved: ahri voice_14_seg0.wav
📁 segment saved: ahri voice_51_seg0.wav
📁 segment saved: ahri voice_13_seg0.wav
📁 segment saved: ahri voice_55_seg0.wav
📁 segment saved: ahri voice_8_seg0.wav
📁 segment saved: ahri voice_46_seg0.wav
📁 segment saved: ahri voice_32_seg0.wav
📁 segment saved: ahri voice_57_seg0.wav
📁 segment saved: ahri voice_37_seg0.wav
📁 segment saved: ahri voice_21_seg0.wav
📁 segment saved: ahri voice_34_seg0.wav
📁 segment saved: ahri voice_3_seg0.wav
📁 segment saved: ahri voice_61_seg0.wav
📁 segment saved: ahri voice_49_seg0.wav
📁 segment saved: ahri voice_18_seg0.wav


100%|██████████| 62/62 [00:00<00:00, 69.57it/s]

📁 segment saved: ahri voice_36_seg0.wav
📁 segment saved: ahri voice_15_seg0.wav
📁 segment saved: ahri voice_56_seg0.wav


In [ ]:
import shutil
from google.colab import files

zip_name = "clips_output.zip"

shutil.make_archive("clips_output", 'zip', "data/clips")

files.download(zip_name)

print("✅ تم تجهيز وتحميل الملف المضغوط.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ تم تجهيز وتحميل الملف المضغوط.
